# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116935e+02     1.493481e+00
 * time: 0.09485292434692383
     1     1.077306e+01     8.391372e-01
 * time: 1.3753299713134766
     2    -1.190959e+01     9.950084e-01
 * time: 1.530095100402832
     3    -3.390107e+01     7.443974e-01
 * time: 1.6728010177612305
     4    -4.741816e+01     5.299625e-01
 * time: 1.8277881145477295
     5    -5.682268e+01     2.097415e-01
 * time: 1.9759950637817383
     6    -5.979251e+01     1.219276e-01
 * time: 2.0918259620666504
     7    -6.091813e+01     4.833103e-02
 * time: 2.2024660110473633
     8    -6.135762e+01     6.115489e-02
 * time: 2.32356595993042
     9    -6.163247e+01     4.399786e-02
 * time: 2.435305118560791
    10    -6.186547e+01     3.175376e-02
 * time: 2.554630994796753
    11    -6.202321e+01     2.227585e-02
 * time: 2.6648881435394287
    12    -6.208308e+01     2.022678e-02
 * time: 2.784420967102051
    13    -6.214002e+01     1.551889e-02
 * time: 2.902368068695068

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557686
    AtomicNonlocal      14.8522650
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666460086
